<a href="https://colab.research.google.com/github/adnil8130/T5LittleGenQA/blob/main/T5%E7%94%9F%E6%88%90%E5%BC%8F%E9%97%AE%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目标
训练一个生成式问答模型，base模型采用Google T5-Base("uer/t5-base-chinese-cluecorpussmall")

预训练模型地址：https://huggingface.co/uer/t5-base-chinese-cluecorpussmall

模型的评价指标采用BLEU-1，BLEU-2，BLEU-3，BLEU-4。

# 数据集
数据集：链接：https://pan.quark.cn/s/6d4a98cd65f2    

提取码：bzne

数据的格式如下：
```
{"context": "违规分为:一般违规扣分、严重违规扣分、出售假冒商品违规扣分,淘宝网每年12月31日24:00点会对符合条件的扣分做清零处理,详情如下:|温馨提醒:由于出售假冒商品24≤N<48分,当年的24分不清零,所以会存在第一年和第二年的不同计分情况。", "answer": "12月31日24:00", "question": "淘宝扣分什么时候清零", "id": 203}
```

# 1. 准备数据

In [58]:
# ! pip install transformers

In [59]:
import transformers
import numpy as np
import torch
print(transformers.__version__)
print(torch.__version__)
from torch.utils.data import Dataset, random_split
import json

4.49.0
2.5.1+cu124


In [60]:
max_context_len = 0
max_question_len = 0
max_answer_len = 0
context = ""
questiont = ""
answer = ""

save_path_file = '.'
train_data_address = save_path_file + '/train.json'
dev_data_address = save_path_file + '/dev.json'

with open(train_data_address, 'rt', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    sample = json.loads(line.strip())
    if len(sample["context"]) > max_question_len:
        max_context_len = len(sample["context"])
        context = sample["context"]
    if len(sample["question"]) > max_question_len:
        max_question_len = len(sample["question"])
        question = sample["question"]
    if len(sample["answer"]) > max_answer_len:
        max_answer_len = len(sample["answer"])
        answer = sample["answer"]

with open(dev_data_address, 'rt', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    sample = json.loads(line.strip())
    if len(sample["context"]) > max_question_len:
        max_context_len = len(sample["context"])
        context = sample["context"]
    if len(sample["question"]) > max_question_len:
        max_question_len = len(sample["question"])
        question = sample["question"]
    if len(sample["answer"]) > max_answer_len:
        max_answer_len = len(sample["answer"])
        answer = sample["answer"]

print("最长context", max_context_len, context)
print("最长question", max_question_len, question)
print("最长answer", max_answer_len, answer)

最长context 107 2015下半年教师资格证考试时间为11月1日，考生可于2015年10月26日—10月31日登录报名系统，根据提示下载pdf准考证文件。下载后，仔细核对个人信息，并直接打印成准考，按准考证上的要求到指定地点参加考试。
最长question 42 痞子猪身上是什么字母? (问题由猫小逗提供)【答题格式为da+答案,例如答案是爱消除
最长answer 110 如果下雨的时候你拖着行李箱子站在屋檐下面那么其实我没有足够的时间找一个好一点的理由抛弃家里面的狗坐上K667次列车到你在的地方找个商店买一把伞然后给我妹妹弹吉他因为她要参加比赛所以我回不去了我也不会给你说我泡面的碗还没洗


## 构建数据集

In [61]:
train_ratio = 0.9
max_dataset_size = 22000
train_set_size = 20000
valid_set_size = 2000

class GenQA(Dataset):
  def __init__(self, data_file):
    self.data = self.load_data(data_file)

  def load_data(self, data_file):
    Data = {}
    with open(data_file, 'rt', encoding='utf-8') as f:
      for idx, line in enumerate(f):
        if idx >= max_dataset_size:
            break
        sample = json.loads(line.strip())
        Data[idx] = sample
    return Data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

data = GenQA(train_data_address)
data_size = len(data)

train_size = int(train_ratio * data_size)
valid_size = data_size - train_size
train_data, valid_data = random_split(data, [train_size, valid_size])
test_data = GenQA(dev_data_address)

In [62]:
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))

train set size: 13068
valid set size: 1452
test set size: 984
{'context': '山村老尸1（香港山村老尸系列恐怖电影）山村老尸的剧情简介······小明（海俊杰）的四个朋友在黄山村玩过一次“招魂游戏”后先后离奇身亡，因直觉事有蹊跷，小明向当记者的姐姐CISSY（黎姿）求助，为获独家新闻，CISSY介绍对灵学颇有研究且暗恋自己的好友发毛（吴镇宇）给小明认识。发毛与小明赶赴黄山村调查时，了解到该村在百年前曾发生过离奇惨案，其时被丈夫密谋杀死后得知自己死因的粤剧名伶楚人美（施念慈）曾在三天内害死该村66条人命，而现在她的阴魂又现是因尸骨被新建的工程抛入山边的潭中。经过更加深入的调查，两人发现女鬼杀人的手法实则是让喝过潭水的人产生幻觉后离奇暴毙，但是此时CISSY及丈夫已经喝过潭水，为救CISSY，小明和发毛先后喝过潭水跳入深潭', 'answer': '1', 'question': '山村老尸楚人美是第几部', 'id': 1003}


## 数据预处理

In [63]:
from transformers import AutoTokenizer
model_checkpoint = './t5-base-chinese-cluecorpussmall'
# model_checkpoint = 'uer/t5-small-chinese-cluecorpussmall'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,return_token_type_ids=False)

In [64]:
context = train_data[0]["context"]
question = train_data[0]["question"]
answer = train_data[0]["answer"]

inputs = tokenizer(context, question)
targets = tokenizer(answer)

In [65]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['[CLS]', '山', '村', '老', '尸', '1', '（', '香', '港', '山', '村', '老', '尸', '系', '列', '恐', '怖', '电', '影', '）', '山', '村', '老', '尸', '的', '剧', '情', '简', '介', '·', '·', '·', '·', '·', '·', '小', '明', '（', '海', '俊', '杰', '）', '的', '四', '个', '朋', '友', '在', '黄', '山', '村', '玩', '过', '一', '次', '[UNK]', '招', '魂', '游', '戏', '[UNK]', '后', '先', '后', '离', '奇', '身', '亡', '，', '因', '直', '觉', '事', '有', '蹊', '跷', '，', '小', '明', '向', '当', '记', '者', '的', '姐', '姐', '[UNK]', '（', '黎', '姿', '）', '求', '助', '，', '为', '获', '独', '家', '新', '闻', '，', '[UNK]', '介', '绍', '对', '灵', '学', '颇', '有', '研', '究', '且', '暗', '恋', '自', '己', '的', '好', '友', '发', '毛', '（', '吴', '镇', '宇', '）', '给', '小', '明', '认', '识', '。', '发', '毛', '与', '小', '明', '赶', '赴', '黄', '山', '村', '调', '查', '时', '，', '了', '解', '到', '该', '村', '在', '百', '年', '前', '曾', '发', '生', '过', '离', '奇', '惨', '案', '，', '其', '时', '被', '丈', '夫', '密', '谋', '杀', '死', '后', '得', '知', '自', '己', '死', '因', '的', '粤', '剧', '名', '伶', '楚', '人', '美', '（', '施', '念', '慈', '）', '曾', '在', '三',

In [66]:
import torch

max_input_length = 256
max_target_length = 128
sample_cnt = 4

inputs = [train_data[s_idx]["context"] + train_data[s_idx]["question"] + "?" for s_idx in range(sample_cnt)]
targets = [train_data[s_idx]["answer"] for s_idx in range(sample_cnt)]

model_inputs = tokenizer(
    inputs,
    padding=True,
    max_length=max_input_length,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False
)
labels = tokenizer(
    text_target=targets,
    padding=True,
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False
)["input_ids"]

end_token_index = torch.where(labels == 102)[-1]
for idx, end_idx in enumerate(end_token_index):
    labels[idx][end_idx+1:] = -100

print('batch_X shape:', {k: v.shape for k, v in model_inputs.items()})
print('batch_y shape:', labels.shape)
print(model_inputs)
print(labels)

batch_X shape: {'input_ids': torch.Size([4, 256]), 'attention_mask': torch.Size([4, 256])}
batch_y shape: torch.Size([4, 10])
{'input_ids': tensor([[ 101, 2255, 3333,  ..., 2797, 3791,  102],
        [ 101,  126, 2189,  ...,    0,    0,    0],
        [ 101, 2792, 3300,  ..., 5445, 1744,  102],
        [ 101, 2207, 5356,  ..., 1377, 1050,  102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}
tensor([[ 101,  122,  102, -100, -100, -100, -100, -100, -100, -100],
        [ 101, 8110,  119,  128,  115,  129,  119,  130, 8341,  102],
        [ 101, 2548, 5143, 7770, 2595, 5543,  102, -100, -100, -100],
        [ 101, 8212,  118, 8202, 1039,  102, -100, -100, -100, -100]])


In [67]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM
import gc

train_batch_size = 64
test_batch_size = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

def clean_cuda(device):
    if device == 'cuda':
        # 清理无用变量
        gc.collect()
        torch.cuda.empty_cache()

clean_cuda(device)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = model.to(device)

def collote_fn(batch_samples):
    batch_inputs, batch_targets = [], []
    for sample in batch_samples:
        batch_inputs.append(sample["context"] + sample["question"] + "?")
        batch_targets.append(sample['answer'])
    batch_data = tokenizer(
        batch_inputs,
        text_target=batch_targets,
        padding=True,
        max_length=max_input_length,
        truncation=True,
        return_tensors="pt",
        return_token_type_ids=False
    )

    # with tokenizer.as_target_tokenizer():
    #     labels = tokenizer(
    #         batch_targets,
    #         padding=True,
    #         max_length=max_target_length,
    #         truncation=True,
    #         return_tensors="pt"
    #     )["input_ids"]

    batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(batch_data['labels'])
    end_token_index = torch.where(batch_data['labels'] == 102)[-1]
    for idx, end_idx in enumerate(end_token_index):
        batch_data['labels'][idx][end_idx+1:] = -100
    return batch_data

train_dataloader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

Using cuda device


In [68]:
batch = next(iter(train_dataloader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print(batch)

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])
batch shape: {'input_ids': torch.Size([64, 256]), 'attention_mask': torch.Size([64, 256]), 'labels': torch.Size([64, 23]), 'decoder_input_ids': torch.Size([64, 23])}
{'input_ids': tensor([[ 101, 2357, 6622,  ...,    0,    0,    0],
        [ 101,  809, 2769,  ...,    0,    0,    0],
        [ 101, 3766,  752,  ..., 5471,  686,  102],
        ...,
        [ 101,  100, 3221,  ...,    0,    0,    0],
        [ 101,  872,  812,  ..., 4495,  772,  102],
        [ 101,  517, 3187,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101,  671, 2094,  ..., -100, -100, -100],
        [ 101,  753,  702,  ..., -100, -100, -100],
        [ 101,  100, 7564,  ..., -100, -100, -100],
        ...,
   

# 2. 模型训练

## 优化模型参数

In [69]:
from tqdm.auto import tqdm
import random

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)

    model.train()
    loss_record_step = []
    for batch, batch_data in enumerate(dataloader, start=1):
        batch_data = batch_data.to(device)
        outputs = model(**batch_data)

        random_number = random.uniform(0, 1)

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        loss_record_step.append(loss.item())
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
        del batch_data, outputs, loss
        clean_cuda(device)
    return total_loss, loss_record_step

## 评估指标

In [70]:
# ! pip install sacrebleu

In [54]:
from sacrebleu.metrics import BLEU

predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
bad_predictions_1 = ["This This This This"]
bad_predictions_2 = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

bleu1 = BLEU(max_ngram_order=1)
print("==========BLEU-1==========")
print(bleu1.corpus_score(predictions, references).score)
print(bleu1.corpus_score(bad_predictions_1, references).score)
print(bleu1.corpus_score(bad_predictions_2, references).score)

bleu2 = BLEU(max_ngram_order=2)
print("==========BLEU-2==========")
print(bleu2.corpus_score(predictions, references).score)
print(bleu2.corpus_score(bad_predictions_1, references).score)
print(bleu2.corpus_score(bad_predictions_2, references).score)

bleu3 = BLEU(max_ngram_order=3)
print("==========BLEU-3==========")
print(bleu3.corpus_score(predictions, references).score)
print(bleu3.corpus_score(bad_predictions_1, references).score)
print(bleu3.corpus_score(bad_predictions_2, references).score)

bleu4 = BLEU(max_ngram_order=4)
print("==========BLEU-4==========")
print(bleu4.corpus_score(predictions, references).score)
print(bleu4.corpus_score(bad_predictions_1, references).score)
print(bleu4.corpus_score(bad_predictions_2, references).score)

==========BLEU-1==========
84.33740467435464
2.634980614046608
0.40867714384640685
==========BLEU-2==========
65.05696445772017
2.1514526621798953
0.40867714384640685
==========BLEU-3==========
53.804523766396294
1.8269935164445736
0.0
==========BLEU-4==========
46.750469682990186
1.683602693167689
0.0


In [55]:
import random

def test_loop(dataloader, model, sample_ovserve_ratio=0.05):
    preds, labels = [], []

    model.eval()
    for batch_data in tqdm(dataloader):
        batch_data = batch_data.to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                batch_data["input_ids"],
                attention_mask=batch_data["attention_mask"],
                max_length=max_target_length,
            ).cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        random_number = random.uniform(0, 1)
        if random_number < sample_ovserve_ratio:
            print("input:", tokenizer.batch_decode(batch_data["input_ids"].cpu().numpy(), skip_special_tokens=True))
            print("output:", decoded_preds)

        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        del batch_data
        clean_cuda(device)

        preds += [pred.strip() for pred in decoded_preds]
        labels += [[label.strip()] for label in decoded_labels]
    return bleu1.corpus_score(preds, labels).score, bleu2.corpus_score(preds, labels).score, bleu3.corpus_score(preds, labels).score, bleu4.corpus_score(preds, labels).score

## 保存模型

In [53]:
# test_data = GenQA(dev_data_address)
# test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

# test_loop(test_dataloader, model)

  0%|          | 0/16 [00:00<?, ?it/s]

(0.0, 0.0, 0.0, 0.0)

In [56]:
import matplotlib.pyplot as plt
import numpy as np

def save_data_and_plot(data, txt_file_path, img_file_path):
    # ===================== 写入文件部分 =====================
    # 追加写入数据（自动创建文件）
    with open(txt_file_path, 'a') as f:
        # 将数字转为字符串并换行写入
        f.write('\n'.join(map(str, data)))
        f.write('\n')  # 添加换行符分隔不同写入批次

    # ===================== 读取文件部分 =====================
    # 从文件读取所有数字
    loaded_data = []
    try:
        with open(txt_file_path, 'r') as f:
            for line in f:
                # 去除空白字符并尝试转换为浮点数
                cleaned_line = line.strip()
                if cleaned_line:
                    loaded_data.append(float(cleaned_line))
    except FileNotFoundError:
        print("错误：文件不存在")
        exit()

    # ===================== 绘图部分 =====================
    plt.figure(figsize=(10, 6))
    plt.plot(loaded_data,
            color='green',
            linestyle='--',
            marker='s',
            markersize=8,
            linewidth=2)

    # 图表装饰
    plt.title("数值变化曲线", fontsize=14, pad=20)
    plt.xlabel("数据索引", fontsize=12, labelpad=10)
    plt.ylabel("测量值", fontsize=12, labelpad=10)
    plt.grid(True, alpha=0.4, linestyle=':')

    # 自动调整坐标轴范围
    plt.xlim(0, len(loaded_data)-1)
    plt.ylim(min(loaded_data)-1, max(loaded_data)+1)

    # 保存和显示
    plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
    plt.close()  # 关闭图表释放内存

    print("操作结果：")
    print(f"- 数据已保存至 {txt_file_path}")
    print(f"- 生成曲线图：{img_file_path}")
    print(f"- 加载数据量：{len(loaded_data)} 条")

In [ ]:
from transformers import AdamW, get_scheduler

learning_rate = 2e-5
epoch_num = 81

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_bleu1 = 0.
best_bleu2 = 0.
best_bleu3 = 0.
best_bleu4 = 0.
best_bleu_weighted_add = 0.
txt_file_path = save_path_file + '/lossdata.txt'
img_file_path = save_path_file + '/lossdata.png'
model_parm_path = None

train_batch_size = 32
test_batch_size = 32

train_dataloader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

if model_parm_path is not None:
    model.load_state_dict(torch.load(model_parm_path))

for t in range(epoch_num):
    clean_cuda(device)
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")

    # 查看清理后剩余显存
    print(f"可用显存: {torch.cuda.mem_get_info()[0]/1024**3:.2f} GB")

    total_loss, loss_record_step = train_loop(train_dataloader, model, optimizer, lr_scheduler, t+1, total_loss)
    save_data_and_plot(loss_record_step, txt_file_path, img_file_path)

    # 查看清理后剩余显存
    print(f"可用显存: {torch.cuda.mem_get_info()[0]/1024**3:.2f} GB")
    valid_bleu1, valid_bleu2, valid_bleu3, valid_bleu4 = test_loop(valid_dataloader, model)
    print(f"BLEU1: {valid_bleu1:>0.2f}\n")
    if valid_bleu1 > best_bleu1:
        best_bleu1 = valid_bleu1
    print(f"BLEU2: {valid_bleu2:>0.2f}\n")
    if valid_bleu2 > best_bleu2:
        best_bleu2 = valid_bleu2
    print(f"BLEU3: {valid_bleu3:>0.2f}\n")
    if valid_bleu3 > best_bleu3:
        best_bleu3 = valid_bleu3
    print(f"BLEU4: {valid_bleu4:>0.2f}\n")
    if valid_bleu4 > best_bleu4:
        best_bleu4 = valid_bleu4

    valid_bleu = 0.1 * valid_bleu1 + 0.2 * valid_bleu2 + 0.3 * valid_bleu3 + 0.4 * valid_bleu4
    if valid_bleu > best_bleu_weighted_add or t % 5 == 0:
        best_bleu_weighted_add = valid_bleu
        print('saving new weights...\n')
        torch.save(model.state_dict(), save_path_file + f'/epoch_{t+1}_loss_{loss_record_step[-1]:0.7f}_valid_bleu_{valid_bleu:0.2f}_model_weights.bin')
print("Done!")

Epoch 1/81
-------------------------------
可用显存: 17.89 GB


  0%|          | 0/409 [00:00<?, ?it/s]

In [ ]:
test_data = GenQA(dev_data_address)
test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

import json

model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.bin'))

model.eval()
with torch.no_grad():
    print('evaluating on test set...')
    sources, preds, labels = [], [], []
    for batch_data in tqdm(test_dataloader):
        batch_data = batch_data.to(device)
        generated_tokens = model.generate(
            batch_data["input_ids"],
            attention_mask=batch_data["attention_mask"],
            max_length=max_target_length,
        ).cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_sources = tokenizer.batch_decode(
            batch_data["input_ids"].cpu().numpy(),
            skip_special_tokens=True,
            use_source_tokenizer=True
        )
        # print('decoded_sources', decoded_sources)

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        # print('decoded_preds', decoded_preds)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)
        # print('decoded_labels', decoded_labels)

        sources += [source.strip() for source in decoded_sources]
        preds += [pred.strip() for pred in decoded_preds]
        labels += [[label.strip()] for label in decoded_labels]
    bleu_score = bleu4.corpus_score(preds, labels).score
    print(f"Test BLEU: {bleu_score:>0.2f}\n")
    results = []
    print('saving predicted results...')
    for source, pred, label in zip(sources, preds, labels):
        results.append({
            "sentence": source,
            "prediction": pred,
            "translation": label[0]
        })
    with open('test_data_pred.json', 'wt', encoding='utf-8') as f:
        for exapmle_result in results:
            f.write(json.dumps(exapmle_result, ensure_ascii=False) + '\n')

In [20]:
model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.bin'))
test_dataloader1 = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collote_fn)
test_loop(test_dataloader1, model, sample_ovserve_ratio=1)


/tmp/ipykernel_1161/442049437.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.b

  0%|          | 0/984 [00:00<?, ?it/s]

input: ['年 基 准 利 率 4. 35 % 。 从 实 际 看, 贷 款 的 基 本 条 件 是 : 一 是 中 国 大 陆 居 民, 年 龄 在 60 岁 以 下 ; 二 是 有 稳 定 的 住 址 和 工 作 或 经 营 地 点 ; 三 是 有 稳 定 的 收 入 来 源 ; 四 是 无 不 良 信 用 记 录, 贷 款 用 途 不 能 作 为 炒 股, 赌 博 等 行 为 ; 五 是 具 有 完 全 民 事 行 为 能 力 。 2017 年 银 行 贷 款 基 准 利 率?']
output: ['']
input: ['年 基 准 利 率 4. 35 % 。 从 实 际 看, 贷 款 的 基 本 条 件 是 : 一 是 中 国 大 陆 居 民, 年 龄 在 60 岁 以 下 ; 二 是 有 稳 定 的 住 址 和 工 作 或 经 营 地 点 ; 三 是 有 稳 定 的 收 入 来 源 ; 四 是 无 不 良 信 用 记 录, 贷 款 用 途 不 能 作 为 炒 股, 赌 博 等 行 为 ; 五 是 具 有 完 全 民 事 行 为 能 力 。 2017 年 银 行 贷 款 基 准 利 率?']
output: ['']
input: ['系 列 是 最 好 的 ， 采 用 国 际 顶 尖 技 术 （ 由 格 力 自 主 研 发 ） 双 级 变 频 压 缩 机 ， 提 高 压 缩 机 运 转 效 率 ， 制 冷 制 热 能 力 更 强 劲 ； 1 赫 兹 变 频 技 术 ， 使 空 调 相 当 于 一 个 15 电 灯 泡 ， 更 加 节 能 省 电 ； 送 风 面 积 广 ， 风 力 大 ； 生 态 风 ， 净 化 空 气 。 非 常 不 错 ， 现 在 国 美 在 做 活 动 ， 可 以 了 解 一 下 。 格 力 空 调 哪 个 系 列 好?']
output: ['']
input: ['平 面 操 作 区 域 进 深 （ 即 宽 度 ） 以 40 至 60 厘 米 为 宜 ； 要 充 分 考 虑 洗 菜 盆 的 宽 度 。 以 标 准 洗 菜 盆 来 算 ， 应 选 择 550 － － 的 宽 度 为 好 。 另 ： 在 高 度 方 面 ， 根 据 我 国 人 体 高 度 测 算 ， 掌 握 以 下 尺 寸 为 宜 ： 操 作 台 高 度 在

KeyboardInterrupt: 